In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [ ]:
import json
import nltk
import time
import spacy
from spacy.matcher import PhraseMatcher
import pandas as pd

# Download necessary NLTK resources
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import requests
import time
import requests
import re
import json
import torch
from torch.nn import CrossEntropyLoss

In [ ]:
# Function to fetch recipes from the Tasty API

def fetch_recipes_by_cuisine(api_key, cuisines, min_recipes_per_cuisine=40, size=20):
    url = "https://tasty.p.rapidapi.com/recipes/list"
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "tasty.p.rapidapi.com"
    }
    all_recipes = []

    for cuisine in cuisines:
        print(f"Fetching recipes for cuisine: {cuisine}")
        recipes = []
        from_index = 0

        while len(recipes) < min_recipes_per_cuisine:
            querystring = {"from": from_index, "size": size, "tags": cuisine}
            try:
                response = requests.get(url, headers=headers, params=querystring)
                if response.status_code == 200:
                    try:
                        data = response.json()
                    except requests.exceptions.JSONDecodeError:
                        print(f"Error decoding JSON for {cuisine}: {response.status_code} - {response.text}")
                        break

                    results = data.get("results", [])
                    if not results:
                        print(f"No more recipes found for cuisine: {cuisine}")
                        break

                    for result in results:
                        name = result.get("name", "Unnamed Recipe")
                        description = result.get("description", "No description available.")
                        ingredients = [ingredient.get("raw_text", "") for ingredient in result.get("sections", [{}])[0].get("components", [])]
                        instructions = [step.get("display_text", "") for step in result.get("instructions", [])]
                        tags = result.get("tags", [])

                        formatted_recipe = {
                            "name": name,
                            "description": description,
                            "ingredients": ingredients,
                            "instructions": instructions,
                            "tags": [tag.get("name", "") for tag in tags]
                        }
                        recipes.append(formatted_recipe)
                        if len(recipes) >= min_recipes_per_cuisine:
                            break

                    from_index += size
                elif response.status_code == 429:
                    print(f"Rate limit hit. Retrying after 10 seconds...")
                    time.sleep(10)
                elif response.status_code == 403:
                    print(f"Access forbidden. Check your API key and subscription.")
                    break
                else:
                    print(f"Unexpected response: {response.status_code} - {response.text}")
                    break

                time.sleep(5)
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
                break

        print(f"Fetched {len(recipes)} recipes for cuisine: {cuisine}")
        all_recipes.extend(recipes)

    return all_recipes

def save_to_json_file(data, filename):
    try:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"Data successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving data to {filename}: {e}")

# Main script
if __name__ == "__main__":
    API_KEY = "3c60d7657fmsh475764087087d73p19bdacjsn588dad9f5412"
    CUISINES = ['chinese', 'mediterranean', 'italian', 'british', 'middle_eastern',
                'korean', 'jewish', 'vietnamese', 'hawaiian', 'thai',
                'caribbean', 'indian', 'brazilian', 'kenyan', 'french',
                'southwestern', 'african', 'japanese', 'mexican', 'north_american', 'european']

    recipes = fetch_recipes_by_cuisine(API_KEY, CUISINES)
    print(f"Total recipes fetched: {len(recipes)}")

    save_to_json_file(recipes, "recipes_dataset.json")


Fetching recipes for cuisine: chinese
Fetched 40 recipes for cuisine: chinese
Fetching recipes for cuisine: mediterranean
Fetched 40 recipes for cuisine: mediterranean
Fetching recipes for cuisine: italian
Fetched 40 recipes for cuisine: italian
Fetching recipes for cuisine: british
Fetched 40 recipes for cuisine: british
Fetching recipes for cuisine: middle_eastern
Unexpected response: 504 - {"messages":"The request to the API has timed out. Please try again later, or if the issue persists, please contact the API provider", "info": "Your Client (working) ---> Gateway (working) ---> API (took too long to respond)"}
Fetched 20 recipes for cuisine: middle_eastern
Fetching recipes for cuisine: korean
Fetched 40 recipes for cuisine: korean
Fetching recipes for cuisine: jewish
Fetched 40 recipes for cuisine: jewish
Fetching recipes for cuisine: vietnamese
No more recipes found for cuisine: vietnamese
Fetched 34 recipes for cuisine: vietnamese
Fetching recipes for cuisine: hawaiian
Fetched 4

In [ ]:
# Load the JSON file
filename = "/content/recipes_dataset.json"
try:
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)

        # Print the first 5 recipes:
        print("First 10 recipes:")
        for i, recipe in enumerate(data[:5], start=1):
            print(f"\nRecipe {i}:")
            print(f"Name: {recipe.get('name', 'N/A')}")
            print(f"Description: {recipe.get('description', 'N/A')}")
            print("Ingredients:")
            for ingredient in recipe.get('ingredients', []):
                print(f"  - {ingredient}")
            print("Instructions:")
            for step in recipe.get('instructions', []):
                print(f"  - {step}")
            print("Tags:")
            for tag in recipe.get('tags', []):
                print(f"  - {tag}")
except FileNotFoundError:
    print(f"Error: The file '{filename}' does not exist.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")


First 10 recipes:

Recipe 1:
Name: Chicken & Veggie Stir-Fry
Description: When you need dinner on the table fast, takeout is often the easiest option. But with our chicken and veggie stir-fry recipe, you can recreate a flavorful dish in the comfort of your own kitchen that's just as fast as takeout. Succulent white meat is perfectly seared to a golden brown, then mixed with mushrooms and broccoli florets in a savory sauce of soy sauce, garlic, ginger, brown sugar, and sesame oil. Served over hot steamed rice or noodles, this is a complete and satisfying meal. The best part? You can customize this recipe based on what's in your fridge, whether swapping the chicken for tofu or shrimp or choosing your favorite mix of veggies. 
Ingredients:
  - 1 pound chicken breast, cubed
  - Salt, to taste
  - Pepper, to taste
  - 1 pound broccoli florets
  - 8 ounces mushrooms, sliced
  - 3 tablespoons oil for frying
Instructions:
  - In a large pan on medium-high heat, add 1 tablespoon of oil.  Once t

In [ ]:
# Parse and preprocess the dataset

def parse_recipes(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    parsed_data = []

    for recipe in data:
        name = recipe.get("name", "").strip() if recipe.get("name") else ""
        description = recipe.get("description", "").strip() if recipe.get("description") else ""
        ingredients = ", ".join(recipe.get("ingredients", [])).strip() if recipe.get("ingredients") else ""
        instructions = " ".join(recipe.get("instructions", [])).strip() if recipe.get("instructions") else ""
        tags = recipe.get("tags", []) if recipe.get("tags") else []

        if name and description and ingredients and tags:
            parsed_data.append({
                "name": name,
                "description": description,
                "ingredients": ingredients,
                "instructions": instructions,
                "tags": tags
            })

    return pd.DataFrame(parsed_data)

# Load and parse recipes
data_path = "recipes_dataset.json"
recipes_df = parse_recipes(data_path)

In [ ]:
recipes_df

,name,description,ingredients,instructions,tags
0,Chicken & Veggie Stir-Fry,"When you need dinner on the table fast, takeou...","1 pound chicken breast, cubed, Salt, to taste,...","In a large pan on medium-high heat, add 1 tabl...","[north_american, chinese, comfort_food, dairy_..."
1,Chicken Teriyaki Chow Mein,"With a medley of colorful veggies, tender chic...","1 pound cubed chicken breast, Salt, to taste, ...","In a medium bowl, season the cubed chicken wit...","[chinese, japanese, comfort_food, dairy_free, ..."
2,Original Orange Chicken by Panda Express,Why order takeout when you can make your own o...,"2 pounds boneless, skinless, chicken thighs, 1...","On a cutting board, cut chicken into 1x1-inch ...","[north_american, chinese, comfort_food, dairy_..."
3,Slow Cooker Beef and Broccoli,This slow cooker beef and broccoli dish lets y...,2 lbs sirloin steak or boneless beef chuck roa...,"In the insert of the crockpot, whisk together ...","[north_american, chinese, comfort_food, dairy_..."
4,Sweet & Sour Chicken,Indulge in this mouthwatering Sweet & Sour Chi...,"1 pound boneless, skinless chicken breasts, cu...",Season chicken with salt and pepper. Coat with...,"[north_american, chinese, comfort_food, dairy_..."
...,...,...,...,...,...
744,Pie Crust,This pie crust recipe is so easy and versatile...,"2 cups all-purpose flour, plus more for dustin...",Sift the flour and salt together into a large ...,"[north_american, british, comfort_food, health..."
745,Butter Cookies,These golden treats are the epitome of simplic...,"1 cup (2 sticks) unsalted butter, softened, 1 ...",Preheat the oven to 350°F (180°C). Line a baki...,"[north_american, comfort_food, easy, christmas..."
746,Chocolate Braided Swirl Bread (Babka),Indulge in this mesmerizing Chocolate Braided ...,"4 cups flour, 2 ¼ teaspoons yeast, 1 cup whole...",Combine the yeast and warm milk and let sit un...,"[comfort_food, christmas, thanksgiving, breakf..."
747,Palmier Cookies Easy Dessert,With their elaborate shape and impressive appe...,"1 sheet puff pastry, thawed, ½ stick melted bu...",Unwrap the puff pastry. Using your hands or a ...,"[north_american, french, dairy_free, 5_ingredi..."


In [ ]:
# unique_tags = set(tag for tags_list in recipes_df["tags"] for tag in tags_list)
# print("Unique Tags:", unique_tags)

In [ ]:
# Filter and preprocess tags

# Define cuisine tags
cuisine_tags = ['chinese', 'mediterranean', 'italian', 'british', 'middle_eastern', 'korean', 'jewish', 'vietnamese', 'hawaiian', 'thai',
                'caribbean', 'indian', 'brazilian', 'kenyan', 'french', 'southwestern', 'african', 'japanese', 'mexican',
                'north_american', 'european']

# Filter recipes to only include cuisine tags
recipes_df["cuisine tags"] = recipes_df["tags"].apply(lambda x: [tag for tag in x if tag in cuisine_tags])

# Remove recipes without any cuisine tag
recipes_df = recipes_df[recipes_df["cuisine tags"].str.len() > 0]

### replace any cuisine_tags that are only "north_american" with "usa"

In [ ]:
recipes_df['cuisine tags'] = recipes_df['cuisine tags'].apply(lambda x: ['american'] if x == ['north_american'] else x)

### explode dataframe to get the cuisine tags to each have their own row

In [ ]:
# Explode the dataframe based on the cuisine tags (if multiple cuisine tags exist, each tag will get its own row)
recipes_df = recipes_df.explode("cuisine tags").reset_index(drop=True)

In [ ]:
# recipes_df["cuisine tags"].value_counts()

### replace any cuisine_tags that are "middle_eastern" with "middle eastern"

In [ ]:
recipes_df['cuisine tags'] = recipes_df['cuisine tags'].replace('middle_eastern', 'middle eastern')

### drop rows with "north_american"

In [ ]:
recipes_df['cuisine tags'] = recipes_df['cuisine tags'].str.strip().str.lower()
recipes_df = recipes_df[recipes_df['cuisine tags'] != 'north_american'].reset_index(drop=True)
recipes_df["cuisine tags"].value_counts()

,count
cuisine tags,
italian,98
european,90
british,88
mexican,83
chinese,76
japanese,59
indian,52
mediterranean,51
french,51


In [ ]:
# recipes_df["description"].iloc[3]

####  put the cuisine (the actual word) inside of the description of the recipe. Add this to the description: {this recipe is of cuisine type: ___}

In [ ]:
recipes_df['description'] = recipes_df.apply(
    lambda row: f"{row['description']} This recipe is of cuisine type: {row['cuisine tags']}." if row['cuisine tags'] else row['description'],
    axis=1
)

In [ ]:
# recipes_df["description"].iloc[100]

## Save the recipes_df as an excel file

In [ ]:
recipes_df.to_excel('/recipes.xlsx', index=False)

In [ ]:
recipes_df

NameError: name 'recipes_df' is not defined

### First create custom rules for the NER model to be able to extract food and cuisine

In [ ]:
# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")


# Create a PhraseMatcher to match specific food items and cuisines
matcher = PhraseMatcher(nlp.vocab)

food_terms = [
    "beef", "pork", "fish", "chicken", "carrot", "apple", "banana", "lettuce", "tomato", "potato",
    "spinach", "onion", "garlic", "broccoli", "cheese", "rice", "egg", "milk", "pasta", "beans",
    "zucchini", "celery", "cucumber", "peach", "strawberry", "kiwi", "mango", "blueberry", "blackberry",
    "avocado", "pumpkin", "squash", "artichoke", "eggplant", "cabbage", "cauliflower", "green_beans",
    "peas", "lentils", "beetroot", "parsnip", "radish", "sweet_potato", "yams", "fennel", "chard",
    "coriander", "parsley", "basil", "oregano", "rosemary", "thyme", "mint", "sage", "bay_leaf",
    "chili", "peppers", "caraway", "cloves", "mustard", "saffron", "turmeric", "cumin", "curry",
    "cardamom", "ginger", "turkey", "lamb", "duck", "tofu", "seitan", "tempeh", "hummus", "salsa",
    "mushrooms", "mayo", "ketchup", "mustard", "mayonnaise", "soy_sauce", "bbq_sauce", "cheddar",
    "mozzarella", "gouda", "brie", "parmesan", "swiss", "cheese_sauce", "pita_bread", "baguette",
    "naan_bread", "croissant", "tortilla", "burrito", "pizza", "burger", "sandwich", "wrap",
    "chicken_wings", "steak", "ribs", "fried_rice", "sushi", "tacos", "fried_egg", "scrambled_eggs",
    "omelette", "waffle", "pancakes", "crepes", "pudding", "donut", "ice_cream", "cheesecake",
    "muffin", "brownie", "cookie", "cake", "cupcake", "pie", "pastry", "scone", "granola",
    "butter", "oil", "vinegar", "syrup", "honey", "soy_sauce", "chocolate", "hot_sauce", "bbq_sauce",
    "teriyaki_sauce", "pesto", "salsa", "tahini", "vinegar", "mustard", "mayonnaise", "marmite",
    "tomato_sauce", "fish_sauce", "mayonnaise", "horseradish", "tzatziki", "worcestershire_sauce",
    "curry_paste", "adobo", "chimichurri", "sriracha", "fruit", "vegetable"
]

cuisine_terms = [
    'chinese', 'mediterranean', 'italian', 'british', 'middle eastern', 'korean', 'jewish', 'vietnamese',
    'hawaiian', 'thai', 'caribbean', 'indian', 'brazilian', 'kenyan', 'french', 'southwestern', 'african',
    'japanese', 'mexican', 'american', 'european']

# Combine both food and cuisine terms
terms = food_terms + cuisine_terms

# Create a list of Span objects for each food and cuisine term
food_and_cuisine_patterns = [nlp.make_doc(term) for term in terms]
matcher.add("FOOD_OR_CUISINE", None, *food_and_cuisine_patterns)

### Begin GPT2 Model

In [ ]:
# Initialize tokenizer and model for GPT-2
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model_gpt2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def load_recipes_from_excel(file_path, sheet_name="Sheet1"):
    """Loads recipe data from an Excel file."""
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    recipes = df.to_dict('records')
    return recipes

def generate_recipe_description(user_input):
    """Generates a recipe description using GPT-2 with few-shot prompt engineering focused on cuisine and ingredients."""
    prompt = (
        "Here are a few examples of recipe requests with corresponding descriptions:\n\n"
        "Example 1:\nRequest: I want a Mexican recipe with beef.\n"
        "Description: A hearty beef taco bowl with guacamole, salsa, and sour cream, perfect for a flavorful Mexican meal.\n\n"
        "Example 2:\nRequest: I need a Thai recipe with chicken.\n"
        "Description: A flavorful Thai green curry made with tender chicken, coconut milk, and fragrant basil.\n\n"
        "Example 3:\nRequest: Can you give me a Korean dish with pork?\n"
        "Description: A spicy and savory Korean BBQ pork dish, served with rice and kimchi for a traditional taste of Korea.\n\n"
        "Now, write a recipe description based on this request:\n"
        f"{user_input}\nDescription: "
    )

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model_gpt2.generate(
        input_ids,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7
    )

    # Decode the output text and return the generated description
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text


def preprocess_text(text):
    """Preprocesses text using NLP techniques."""
    if text is None:
        return ""
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens
              if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)



In [ ]:
def extract_entities(user_input):
    """Extracts food and cuisine entities using SpaCy with custom matching."""
    doc = nlp(user_input)

    # Apply the matcher to find food and cuisine entities
    matches = matcher(doc)
    found_food_and_cuisine = [doc[start:end].text for match_id, start, end in matches]

    # Print the entities SpaCy finds, and our custom food and cuisine items
    print(f"Entities found in the user input: {[ent.text for ent in doc.ents] + found_food_and_cuisine}\n")

    # Create a dictionary to store the results
    entities = {}

    # Filter entities and assign labels: "FOOD" for food terms, and "GPE" for cuisine terms
    for item in found_food_and_cuisine:
        if item in food_terms:
            entities.setdefault('FOOD', []).append(item)
        elif item in cuisine_terms:
            entities.setdefault('GPE', []).append(item)

    # Also check entities recognized by SpaCy
    for ent in doc.ents:
        if ent.text not in found_food_and_cuisine:  # Avoid duplicates from custom matcher
            if ent.label_ == 'GPE' and ent.text in cuisine_terms:
                entities.setdefault('GPE', []).append(ent.text)
            elif ent.label_ == 'FOOD' and ent.text in food_terms:
                entities.setdefault('FOOD', []).append(ent.text)

    return entities



def tfidf_match(user_input, recipes):
    """Matches user input with recipe descriptions using TF-IDF."""
    vectorizer = TfidfVectorizer()

    # Combine user input with all recipe descriptions
    texts = [user_input] + [recipe.get("description", "") for recipe in recipes]
    tfidf_matrix = vectorizer.fit_transform(texts)

    # Compute cosine similarity between user input and all recipes
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    # Pair recipes with their similarity scores
    recipe_scores = list(zip(recipes, similarities))

    # Sort recipes by similarity in descending order
    recipe_scores = sorted(recipe_scores, key=lambda x: x[1], reverse=True)
    return recipe_scores

def prioritize_by_entities(user_input, recipe_scores):
  """Boosts scores for recipes that match specific entities in the user input."""
  entities = extract_entities(user_input)
  boosted_scores = []

  for recipe, score in recipe_scores:
      description = recipe.get("description", "").lower()

      # Directly check if any FOOD or GPE entities are in the description
      food_matches = [food_item.lower() for food_item in entities.get("FOOD", []) if food_item.lower() in description]
      cuisine_matches = [cuisine.lower() for cuisine in entities.get("GPE", []) if cuisine.lower() in description]

      # Boost the score by 0.12 for a match by cuisine and by 0.10 for each match by food
      score += 0.1 * len(food_matches) + 0.12 * len(cuisine_matches)

      # Append the recipe with the updated score
      boosted_scores.append((recipe, score))

  boosted_scores = sorted(boosted_scores, key=lambda x: x[1], reverse=True)

  return boosted_scores

def get_recipe_recommendations(user_input, recipes):
    """Provides recipe recommendations based on user input."""
    generated_description = generate_recipe_description(user_input)

    # TF-IDF matching for recommendations
    recipe_scores = tfidf_match(user_input, recipes)

    # Print recipe scores before boosting
    print("\nRecipe Similarity Scores Before Boosting:")
    for i, (recipe, score) in enumerate(recipe_scores[:5]):
        print(f"Rank {i + 1}: {recipe.get('name', 'No name')} - Score: {score:.4f}")

    # Boost scores based on specific entities
    boosted_recipe_scores = prioritize_by_entities(user_input, recipe_scores)

    # Print recipe scores after boosting
    print("\nTop 5 Recipe Similarity Scores after Boosting:")
    for i, (recipe, score) in enumerate(boosted_recipe_scores[:5]):
        print(f"Rank {i + 1}: {recipe.get('name', 'No name')} - Score: {score:.4f}")

    # Return the top recommendation based on boosted TF-IDF similarity
    recommended_recipe = boosted_recipe_scores[0][0] if boosted_recipe_scores else None

    reason = (
        f"This recipe was chosen because it closely matches your input.\n"
    )

    return recommended_recipe, reason


recipes = load_recipes_from_excel("/recipes.xlsx")

user_inputs = [
    "Give me a mexican beef recipe",
    "I'm looking for an Italian recipe that will impress my friends",
    "I want a japanese recipe with meat",
    "I need a recipe with chicken and vegetables",
    "I need a Thai recipe"
]

for user_input in user_inputs:
    recommendation, reason = get_recipe_recommendations(user_input, recipes)

    print(f"User Input: {user_input}")
    print(f"Recommended Recipe: {recommendation.get('name', 'No recommendation found.')}")
    print(reason)

    # --- Print details from DataFrame columns ---
    print("Ingredients:")
    ingredients_column = "ingredients"
    ingredients_list = recommendation.get(ingredients_column, "")
    if isinstance(ingredients_list, str):  # Check if ingredients are a string
        ingredients_list = ingredients_list.split(', ')  # Split into a list if needed
    for ingredient in ingredients_list:
        print(f"  - {ingredient}")

    print("Instructions:")
    instructions_column = "instructions"
    instructions_text = recommendation.get(instructions_column, "")
    if isinstance(instructions_text, str):  # Check if instructions are a string
        instructions_list = instructions_text.split('. ')  # Split into steps if needed
        for instruction in instructions_list:
            print(f"  - {instruction.strip()}")  # Strip extra whitespace
    # -------------------------------------------

    print("-" * 20)  # Separator between recommendations


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Recipe Similarity Scores Before Boosting:
Rank 1: Vietnamese Coffee - Score: 0.1795
Rank 2: One-Pot Mexican Quinoa - Score: 0.1593
Rank 3: Thai Iced Tea Caffeinated Smoothie - Score: 0.1541
Rank 4: Crispy Chili Beef - Score: 0.1522
Rank 5: Crispy Chili Beef - Score: 0.1522
Entities found in the user input: ['mexican', 'mexican', 'beef']


Top 5 Recipe Similarity Scores after Boosting:
Rank 1: Southwestern Taco Salad - Score: 0.3468
Rank 2: Cauliflower “Meat” Nachos - Score: 0.3468
Rank 3: Southwest Stuffed Acorn Squash - Score: 0.3463
Rank 4: One-Pot Taco Spaghetti - Score: 0.3386
Rank 5: Taco-Stuffed Peppers - Score: 0.3247
User Input: Give me a mexican beef recipe
Recommended Recipe: Southwestern Taco Salad
This recipe was chosen because it closely matches your input.

Ingredients:
  - 1/2 an avocado
  - 1/2 cup chopped cilantro
  - 1 Tablespoon lime juice
  - 1/2 cup plain low fat greek yogurt
  - 1/3 cup water
  - 1/2 tsp salt
  - 1/2 Tablespoon garlic
  - 1/2 Tablespoon cumin
Ins

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Recipe Similarity Scores Before Boosting:
Rank 1: One-Pot Chicken Spinach Bacon Alfredo - Score: 0.2586
Rank 2: Baked Brie In A Bread Bowl - Score: 0.2246
Rank 3: Kanafeh - Score: 0.1849
Rank 4: Kanafeh - Score: 0.1846
Rank 5: Kanafeh - Score: 0.1841
Entities found in the user input: ['Italian']


Top 5 Recipe Similarity Scores after Boosting:
Rank 1: One-Pot Chicken Spinach Bacon Alfredo - Score: 0.2586
Rank 2: Baked Brie In A Bread Bowl - Score: 0.2246
Rank 3: Kanafeh - Score: 0.1849
Rank 4: Kanafeh - Score: 0.1846
Rank 5: Kanafeh - Score: 0.1841
User Input: I'm looking for an Italian recipe that will impress my friends
Recommended Recipe: One-Pot Chicken Spinach Bacon Alfredo
This recipe was chosen because it closely matches your input.

Ingredients:
  - 6 strips bacon
  - chopped
  - 2 chicken breasts
  - sliced
  - 5 ounces spinach
  - 4 cloves garlic
  - chopped
  - 2 teaspoons salt
  - 1 teaspoon pepper
  - 5 cups milk
  - 1 pound dry penne pasta
  - 1 cup parmesan cheese
  - g

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Recipe Similarity Scores Before Boosting:
Rank 1: Teriyaki Baked Wings - Score: 0.2407
Rank 2: Teriyaki Chicken Fried Rice Dome - Score: 0.2335
Rank 3: Karaage Fried Chicken - Score: 0.1857
Rank 4: Ramen Carbonara - Score: 0.1771
Rank 5: Beef & Broccoli Fried Rice - Score: 0.1594
Entities found in the user input: ['japanese', 'japanese']


Top 5 Recipe Similarity Scores after Boosting:
Rank 1: Teriyaki Baked Wings - Score: 0.3607
Rank 2: Teriyaki Chicken Fried Rice Dome - Score: 0.3535
Rank 3: Karaage Fried Chicken - Score: 0.3057
Rank 4: Ramen Carbonara - Score: 0.2971
Rank 5: Beef & Broccoli Fried Rice - Score: 0.2794
User Input: I want a japanese recipe with meat
Recommended Recipe: Teriyaki Baked Wings
This recipe was chosen because it closely matches your input.

Ingredients:
  - 1½ pounds chicken wings
  - 2 teaspoons baking powder
  - ¼ teaspoon salt
Instructions:
  - Preheat oven to 250˚F (130˚C)
  - Place wings in a large bowl and pat dry
  - Add the baking powder and salt to

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Recipe Similarity Scores Before Boosting:
Rank 1: Sweet-and-sour Chicken Noodles - Score: 0.2650
Rank 2: Karaage Fried Chicken - Score: 0.2317
Rank 3: Creamy Chicken Penne Pasta - Score: 0.2274
Rank 4: Creamy Chicken Penne Pasta - Score: 0.2274
Rank 5: Healthy Veggies and Couscous - Score: 0.2190
Entities found in the user input: ['chicken']


Top 5 Recipe Similarity Scores after Boosting:
Rank 1: Sweet-and-sour Chicken Noodles - Score: 0.3650
Rank 2: Karaage Fried Chicken - Score: 0.3317
Rank 3: Creamy Chicken Penne Pasta - Score: 0.3274
Rank 4: Creamy Chicken Penne Pasta - Score: 0.3274
Rank 5: Chicken Teriyaki Meatballs - Score: 0.3133
User Input: I need a recipe with chicken and vegetables
Recommended Recipe: Sweet-and-sour Chicken Noodles
This recipe was chosen because it closely matches your input.

Ingredients:
  - 2 breasts of chicken
  - 1 teaspoon salt
  - ½ teaspoon pepper
  - 2 cloves garlic
  - 1 red pepper
  - 2 tablespoons soy sauce
  - 1 tablespoon ketchup
  - 1 tables